In [ ]:
import dfxml.fiwalk as fiwalk
import dfxml 
import sys
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import pprint
from pprint import pprint
imagefile = open("/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.raw")
data = ""
def process(fi): # fi=fileobject
    #offset = fi.contents().find(data)
    #if offset>0:
    print(fi.filename())
    
objs = fiwalk.fileobjects_using_sax(imagefile=imagefile)
print(len(objs))


dfxml_path = "/Volumes/Samsung_T5/M57/pat-2009-12-10.xml"
img_path = "/Volumes/Samsung_T5/argo/pat2.csv"

id = 27964
#Lets make sure all "files" are covered
# Lets store it a DB



files_cols = ['obj_id', 'partition','inode','filename','filesize']
block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
file_df = pd.DataFrame(columns=files_cols)
block_hashes_df = pd.DataFrame(columns=block_hashes_cols)

sector_size = 512

counter = 0
print(len(objs))
#with open(img_path, 'rb') as img_h:
for obj in objs:
    if int(obj._tags['id']) == id:# and obj.filesize() == False:
        counter += 1
        #print(obj.filename())
        data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()]
        file_df.loc[len(file_df.index)] = data
        print(data)
        byterun = []
        persist_img_offset = 0
        persist_fs_offset = 0
        persist_file_offset = 0
        print(obj.byte_runs())
        for byterun1 in obj.byte_runs():
            pprint(vars(byterun1))


In [30]:
        print(len(obj.byte_runs()))

#for ebyterun in obj.byte_runs():
#    pprint(vars(ebyterun))


1


In [74]:
id = 11580
#Lets make sure all "files" are covered
# Lets store it a DB
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import pprint
import math
dfxml_path = "/Volumes/Samsung_T5/M57/pat-2009-12-10.xml"
img_csv = "/Volumes/Samsung_T5/argo/pat2.csv"
img_csv_df = pd.read_csv(img_csv)
img_path =  "/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.raw"
files_cols = ['obj_id', 'partition','inode','filename','filesize']
block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
file_df = pd.DataFrame(columns=files_cols)
block_hashes_df = pd.DataFrame(columns=block_hashes_cols)

sector_size = 512
from pprint import pprint
counter = 0
print(len(objs))
with open(img_path, 'rb') as img_h:
    for obj in objs:
        if int(obj._tags['id']) == id:# and obj.filesize() == False:
            counter += 1
            #print(obj.filename())
            data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()]
            file_df.loc[len(file_df.index)] = data
            print(data)
            byterun = []
            persist_img_offset = 0
            persist_fs_offset = 0
            persist_file_offset = 0
            remaining_len = obj.filesize()
            ebyterun_df = pd.DataFrame(columns=['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1'])
            for ebyterun in obj.byte_runs():
                #pprint(vars(ebyterun)) 
                #byterun.append([ebyterun.img_offset,ebyterun.file_offset, ebyterun.len, ebyterun.fs_offset])
                byterun.append(ebyterun.file_offset)
                persist_file_offset = ebyterun.file_offset
                byterun.append(ebyterun.len)
                if hasattr(ebyterun,'img_offset'): 
                    byterun.append(ebyterun.img_offset)
                    if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
                if hasattr(ebyterun,'fs_offset'): 
                    byterun.append(ebyterun.fs_offset)
                    if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset
                if hasattr(ebyterun,'len'): 
                    byterun.append(ebyterun.len)
                    if ebyterun.len != None: persist_len = ebyterun.len
                #print("img_offset", img_offset, "file offset", ebyterun.file_offset)
                #print("file_offset", "len", "img_offset", "fs_offset")
                byterun_start = int(persist_img_offset / sector_size)
                byterun_end = int((math.ceil((persist_img_offset + ebyterun.len) / sector_size))) - 1
                len_run = np.arange(persist_len,0,-sector_size) #remaining_len
                sector_run = np.full(len(len_run), sector_size)
                len_run = np.minimum(len_run,sector_run)
                #len_run = min(sector_size,len_run.all())
                print("len_run",len(len_run), len(len_run)*sector_size)
                print("byterun_start", byterun_start,persist_img_offset, "byterun_end", byterun_end,  "len", ebyterun.len)
                print("ebyterun_df", ebyterun_df.shape, "img_csv", img_csv_df.loc[byterun_start:byterun_end,'img_offset'].shape )
                #print(img_csv_df.loc[byterun_start:byterun_end,:])
                print("img_csv_len",len(img_csv_df.loc[byterun_start:byterun_end, :]))
                ebyterun_df.loc[:,'img_sector_offset'] = np.arange(len(img_csv_df))
                ebyterun_df.loc[byterun_start:byterun_end,'obj_id'] = obj._tags['id']
                ebyterun_df.loc[byterun_start:byterun_end,'img_offset'] = img_csv_df.loc[byterun_start:byterun_end,'img_offset']
                ebyterun_df.loc[byterun_start:byterun_end,'fs_offset'] = np.arange(persist_fs_offset,persist_fs_offset+ebyterun.len,sector_size)
                ebyterun_df.loc[byterun_start:byterun_end,'file_offset'] = np.arange(persist_file_offset,persist_file_offset+ebyterun.len, sector_size)
                ebyterun_df.loc[byterun_start:byterun_end,'len'] = len_run
                #print("remaining_len",remaining_len)
                #remaining_len-=sector_size
                ebyterun_df.loc[byterun_start:byterun_end,'md5'] = img_csv_df.loc[byterun_start:byterun_end,'md5']
            print(ebyterun_df)    
'''
                remaining_len = ebyterun.len
                #cur_file_offset = 0
                #print("image offset", persist_img_offset)
                #print("file offset", ebyterun.file_offset)
                #print("len", ebyterun.len)
                #print()
                for pointer in range(persist_img_offset,persist_img_offset+ebyterun.len, sector_size):
                    #print(pointer)
                    persist_img_offset = pointer
                    
                    img_h.seek(pointer)
                    fsector = img_h.read(sector_size)
                    
                    hash_len = min(remaining_len,sector_size)
                    
                    sector_md5 = hashlib.md5(fsector).hexdigest()
                    sector_sha1 = hashlib.sha1(fsector).hexdigest()
                    data2 = [obj._tags['id'], pointer, persist_fs_offset, persist_file_offset, hash_len, sector_md5, sector_sha1]
                    persist_fs_offset+=sector_size 
                    persist_file_offset+=sector_size 
                    remaining_len -= sector_size
                    block_hashes_df.loc[len(block_hashes_df.index)] = data2

                print(counter, " out of ", len(objs), "id= ",obj._tags['id'])

    #print(len(file_df))
conn = sqlite3.connect("/Users/seunfuta/Downloads/M57/charlie-2009-12-11.db")
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS files (obj_id INTEGER, partition INTEGER, inode INTEGER, filename TEXT, filesize INTEGER)')
conn.commit()
cur.execute('CREATE TABLE IF NOT EXISTS block_hashes (obj_id INTEGER, img_offset INTEGER,fs_offset INTEGER, file_offset INTEGER,len INTEGER, md5 TEXT,sha1 TEXT)')
conn.commit()
file_df.to_sql('files', conn, if_exists='replace', index = False)
block_hashes_df.to_sql('block_hashes', conn, if_exists='replace', index = False)
conn.close()
'''

49745
['11580', '1', '24051', 'Documents and Settings/Pat/Local Settings/Temporary Internet Files/Content.IE5/O7EPGR2J/Firefox%20Setup%203.5.5[1].exe', 8084968]
len_run 32 16384
byterun_start 4423231 2264694272 byterun_end 4423262 len 16384
ebyterun_df (0, 7) img_csv (32,)
img_csv_len 32
len_run 64 32768
byterun_start 10277783 5262224896 byterun_end 10277846 len 32768
ebyterun_df (25429824, 8) img_csv (64,)
img_csv_len 64
len_run 160 81920
byterun_start 13496775 6910348800 byterun_end 13496934 len 81920
ebyterun_df (25429824, 8) img_csv (160,)
img_csv_len 160
len_run 256 131072
byterun_start 12483679 6391643648 byterun_end 12483934 len 131072
ebyterun_df (25429824, 8) img_csv (256,)
img_csv_len 256
len_run 256 131072
byterun_start 11097559 5681950208 byterun_end 11097814 len 131072
ebyterun_df (25429824, 8) img_csv (256,)
img_csv_len 256
len_run 256 131072
byterun_start 11025599 5645106688 byterun_end 11025854 len 131072
ebyterun_df (25429824, 8) img_csv (256,)
img_csv_len 256
len_run 

'\n                remaining_len = ebyterun.len\n                #cur_file_offset = 0\n                #print("image offset", persist_img_offset)\n                #print("file offset", ebyterun.file_offset)\n                #print("len", ebyterun.len)\n                #print()\n                for pointer in range(persist_img_offset,persist_img_offset+ebyterun.len, sector_size):\n                    #print(pointer)\n                    persist_img_offset = pointer\n                    \n                    img_h.seek(pointer)\n                    fsector = img_h.read(sector_size)\n                    \n                    hash_len = min(remaining_len,sector_size)\n                    \n                    sector_md5 = hashlib.md5(fsector).hexdigest()\n                    sector_sha1 = hashlib.sha1(fsector).hexdigest()\n                    data2 = [obj._tags[\'id\'], pointer, persist_fs_offset, persist_file_offset, hash_len, sector_md5, sector_sha1]\n                    persist_fs_offse

In [62]:
ebyterun_df.loc[13558039:13558046,:]


,obj_id,img_offset,fs_offset,file_offset,len,md5,sha1,img_sector_offset
13558039,1211,6941715968,6941683712,0,512,39dcab5c1d131f2a84db06959ae4a4d7,NaN,13558039
13558040,1211,6941716480,6941684224,512,512,111ef107791fd7ab654a04e91135a072,NaN,13558040
13558041,1211,6941716992,6941684736,1024,512,dbf0e745da11cd5810f2189cc2587534,NaN,13558041
13558042,1211,6941717504,6941685248,1536,512,274c3ccd287a94479a60414a1686cd42,NaN,13558042
13558043,1211,6941718016,6941685760,2048,512,227b5a49b52935d27e6f2ed538161d6e,NaN,13558043
13558044,1211,6941718528,6941686272,2560,512,7d67abbcd8d5c7b03d6006bf00f7562a,NaN,13558044
13558045,1211,6941719040,6941686784,3072,512,90af5b3c60b4ccd524ade98ecb5f0691,NaN,13558045
13558046,1211,6941719552,6941687296,3584,121,fb6f3f4d96419ea9b887ca9defa28911,NaN,13558046


In [55]:
ebyterun_df.loc[13558039:13558046,:]

,obj_id,img_offset,fs_offset,file_offset,len,md5,sha1,img_sector_offset
13558039,1211,6941715968,6941683712,0,True,39dcab5c1d131f2a84db06959ae4a4d7,NaN,13558039
13558040,1211,6941716480,6941684224,512,True,111ef107791fd7ab654a04e91135a072,NaN,13558040
13558041,1211,6941716992,6941684736,1024,True,dbf0e745da11cd5810f2189cc2587534,NaN,13558041
13558042,1211,6941717504,6941685248,1536,True,274c3ccd287a94479a60414a1686cd42,NaN,13558042
13558043,1211,6941718016,6941685760,2048,True,227b5a49b52935d27e6f2ed538161d6e,NaN,13558043
13558044,1211,6941718528,6941686272,2560,True,7d67abbcd8d5c7b03d6006bf00f7562a,NaN,13558044
13558045,1211,6941719040,6941686784,3072,True,90af5b3c60b4ccd524ade98ecb5f0691,NaN,13558045
13558046,1211,6941719552,6941687296,3584,True,fb6f3f4d96419ea9b887ca9defa28911,NaN,13558046


In [ ]:
        '''
        for ebyterun in obj.byte_runs():
            pprint(vars(ebyterun)) 
        
            byterun.append(ebyterun.file_offset)
            persist_file_offset = ebyterun.file_offset
            byterun.append(ebyterun.len)
            if hasattr(ebyterun,'img_offset'): 
                byterun.append(ebyterun.img_offset)
                if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
            if hasattr(ebyterun,'fs_offset'): 
                byterun.append(ebyterun.fs_offset)
                if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset

            
            remaining_len = ebyterun.len

            for pointer in range(persist_img_offset,persist_img_offset+ebyterun.len, sector_size):
                #print(pointer)
                persist_img_offset = pointer
                
                img_h.seek(pointer)
                fsector = img_h.read(sector_size)
                
                hash_len = min(remaining_len,sector_size)
                
                sector_md5 = hashlib.md5(fsector).hexdigest()
                sector_sha1 = hashlib.sha1(fsector).hexdigest()
                data2 = [obj._tags['id'], pointer, persist_fs_offset, persist_file_offset, hash_len, sector_md5, sector_sha1]
                persist_fs_offset+=sector_size 
                persist_file_offset+=sector_size 
                remaining_len -= sector_size
                block_hashes_df.loc[len(block_hashes_df.index)] = data2
            
            print(counter, " out of ", len(objs), "id= ",obj._tags['id'])
        '''

In [ ]:
        '''
#print(len(file_df))
'''
conn = sqlite3.connect("/Volumes/Samsung_T5/M57/pat-2009-12-10.db")
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS files (obj_id INTEGER, partition INTEGER, inode INTEGER, filename TEXT, filesize INTEGER)')
conn.commit()
cur.execute('CREATE TABLE IF NOT EXISTS block_hashes (obj_id INTEGER, img_offset INTEGER,fs_offset INTEGER, file_offset INTEGER,len INTEGER, md5 TEXT,sha1 TEXT)')
conn.commit()
file_df.to_sql('files', conn, if_exists='replace', index = False)
block_hashes_df.to_sql('block_hashes', conn, if_exists='replace', index = False)
conn.close()
'''